In [64]:

import pandas as pd
import requests
from tqdm import tqdm
import matplotlib.pyplot as plt
import datetime

In [81]:

def getWormholeData(ite):
    """
    Get Wormhole Token Transfers
    Parameter: ite: number of iterations to loop through - 50 tx per iteration
    """
    df = pd.DataFrame()
    for i in tqdm(range(ite)):
        try:
            url = f'https://api.wormholescan.io/api/v1/transactions/?page={ite}&pageSize=1000'
            df_ = pd.DataFrame(requests.get(url).json()['transactions'])
            df = pd.concat([df, df_])
        except Exception as e:
            print(e)
            print(requests.get(url).json())
            return 
    
    # Reshape the DataFrame
    try:
        reshapedDf =  pd.concat([pd.json_normalize(df['globalTx']).set_index(df.index),
                pd.json_normalize(df["standardizedProperties"]).set_index(df.index)],
                axis =1)

        df_final = pd.concat([df, reshapedDf], axis=1)
        
        def replace_chain_ids(df, column_name):
            df[column_name] = df[column_name].map(wormhole_chain_ids).fillna("Unknown")
            return df
        
        df_final = replace_chain_ids(df_final, "emitterChain")
        df_final = replace_chain_ids(df_final, "toChain")
        df_final = replace_chain_ids(df_final, "tokenChain")
        
        return df_final
    
    except Exception as e: 
        print(e)
        return df 
    

# Wormhole Chain ID Mapping Dictionary
wormhole_chain_ids = {
    1: "Solana", 2: "Ethereum", 4: "BNB Smart Chain",
    5: "Polygon", 6: "Avalanche", 7: "Oasis",
    8: "Algorand", 10: "Fantom", 11: "Karura",
    12: "Acala", 13: "Klaytn", 14: "Celo",
    15: "NEAR", 16: "Moonbeam", 17: "Neon",
    18: "Terra 2", 19: "Injective", 20: "Osmosis",
    22: "Aptos", 23: "Arbitrum", 24: "Optimism",
    25: "Gnosis", 26: "Sui", 28: "PythNet",
    29: "XPLA", 30: "Base", 32: "Sei",
    3104: "Wormchain", 10002: "Sepolia Testnet", 10004: "BSC Testnet",
    10005: "Mumbai Testnet", 10006: "Fuji Testnet", 10010: "Fantom Testnet",
    10016: "Moonbase Alpha Testnet", 10023: "Arbitrum Goerli", 10024: "Optimism Goerli"
}

In [34]:
df = getWormholeData(1000)

df.drop([
    'emitterAddress', 'id', 'txHash', 'originTx.attribute.value.originTxHash',
    'payload', 'standardizedProperties', 'globalTx', 'originTx.txHash', 'fromAddress',
    'fromChain', 'normalizedDecimals', 'originTx.attribute.value.originChainId', 'feeAddress',
    'feeChain', 'originTx.status','originTx.attribute', 'originTx', 'originTx.attribute.type',
    'originTx.attribute.value.originAddress', 'amount'], axis=1, inplace=True)

df.loc[df.tokenAmount.astype(float) > 0]

df['timestamp_'] = pd.to_datetime(df['timestamp']).dt.floor('T')
df['tx_time'] = pd.to_datetime(df['destinationTx.timestamp']) - pd.to_datetime(df['timestamp'])
df = df.loc[df['tx_time']> datetime.timedelta(days=0)]
tx_time = df.groupby('timestamp_').tx_time.mean()
plt.plot(tx_time.index, pd.to_timedelta(tx_time.values).total_seconds())
plt.yscale('log')


,timestamp,emitterChain,emitterNativeAddress,tokenAmount,usdAmount,symbol,destinationTx,originTx.from,destinationTx.chainId,destinationTx.status,...,destinationTx.to,destinationTx.blockNumber,destinationTx.timestamp,destinationTx.updatedAt,appIds,fee,toAddress,toChain,tokenAddress,tokenChain
1,2024-12-20T19:19:23Z,Ethereum,0x3ee18b2214aff97000d974cf647e7c347e8fa585,0.148725,512.47214325,WETH,NaN,0xe71f8d508e1306ea6b47925e42c7f17e0d078612,1.0,completed,...,wormDTUJ6AWPNvk59vGQbDvGJmqbDTdgWgAqcLBCgUb,308732937,2024-12-20T19:35:09Z,2024-12-20T19:36:00.724628Z,[PORTAL_TOKEN_BRIDGE],0,EguycyPF6rQ4znP2KhcvKpLx1QcrYa4RsMdFAX6zxXHX,Solana,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,Ethereum
2,2024-12-20T19:19:23Z,Ethereum,0xbf5f3f65102ae745a48bd521d10bab5bf02a9ef4,2.52266732,482.0312715,SOL,NaN,0xe71f8d508e1306ea6b47925e42c7f17e0d078612,1.0,completed,...,FC4eXxkyrMPTjiYUpp4EAnkmwMbQyZ6NDCh1kfLn6vsf,308732972,2024-12-20T19:35:23Z,2024-12-20T19:36:49.400843Z,[MAYAN],,GVjYEJ7bJiupUzbsUgwxfGhwHyYs16jaZ6VEm1ufpLA,Solana,So11111111111111111111111111111111111111112,Solana
4,2024-12-20T19:19:09Z,Unknown,0xc57508ee0d4595e5a8728974a4a93a787d38f3397572...,76218.459998,76218.459998,USDT,NaN,0x058691491ff4aff14ad074811c91a83fb42cd32d0b32...,1.0,completed,...,wormDTUJ6AWPNvk59vGQbDvGJmqbDTdgWgAqcLBCgUb,308757873,2024-12-20T22:29:45Z,2024-12-20T22:31:43.822677Z,[PORTAL_TOKEN_BRIDGE],0,3TAb4ksgbf5hNZxzwCD97W8BeGwVSk67j2PJyimYgbiX,Solana,0xdAC17F958D2ee523a2206206994597C13D831ec7,Ethereum
5,2024-12-20T19:19:01Z,Solana,5JGpVmGF976mzhzQrfk2BSwqG8b2xg4NxQ88upmZ3iyR,20000,1538,L3,NaN,7xtSuVxZeBVrqjpkY9MuPJW1C9hvdFNbuBye9TL77eXK,24.0,completed,...,0x8d77ac62a6571a408e5c9655ff5de90d537c3045,129561793,2024-12-20T19:19:23Z,2024-12-20T19:41:30.890987Z,[NATIVE_TOKEN_TRANSFER],,0xc2E71028351E51A695D313b11D0AFB99D5931a63,Optimism,5k84VjAKoGPXa7ias1BNgKUrX7e61eMPWhZDqsiD4Bpe,Solana
8,2024-12-20T19:18:35Z,BNB Smart Chain,0xb6f6d86a8f9879a9c87f643768d9efc38c1da6e7,69709.41614161,429.76482436,ICE,NaN,0x8678f58ac6c4748b5289d0db70e627eef395dead,23.0,completed,...,0x0b2402144bb366a632d14b83f244d2e0e21bd39c,286832136,2024-12-20T19:18:52Z,2024-12-20T19:35:47.038798Z,[PORTAL_TOKEN_BRIDGE],0,0x8678F58AC6c4748b5289D0Db70e627eef395deaD,Arbitrum,0xc335Df7C25b72eEC661d5Aa32a7c2B7b2a1D1874,BNB Smart Chain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,2024-12-20T19:00:10Z,Solana,Gv1KWf8DT1jKv5pKBmGaTmVszqa56Xn8YGx2Pg7i7qAk,1100,4939,SUI,NaN,AerVhDV6fqaJrjXvLoU3fFwdrdD58YC17FkzZYA6RPTx,21.0,completed,...,,92594581,2024-12-20T19:01:21Z,2024-12-20T19:01:25.330465Z,"[PORTAL_TOKEN_BRIDGE, CONNECT]",11231005,0xdeada6b862292d44ad20701e51040fa655e95e30b2e3...,Unknown,0x9258181f5ceac8dbffb7030890243caed69a9599d288...,Unknown
976,2024-12-20T18:59:35Z,Ethereum,0xbf5f3f65102ae745a48bd521d10bab5bf02a9ef4,0.24928059,47.63253513,SOL,NaN,0x65519ef77c82302669b843a0b8b5f87077e286a2,1.0,completed,...,FC4eXxkyrMPTjiYUpp4EAnkmwMbQyZ6NDCh1kfLn6vsf,308730236,2024-12-20T19:16:13Z,2024-12-20T19:17:29.644153Z,[MAYAN],,3Ud4Y8B76F43Be6hskj8cymAexZ6u4efDJQpQP6ScEXM,Solana,So11111111111111111111111111111111111111112,Solana
978,2024-12-20T18:59:35Z,Ethereum,0x3ee18b2214aff97000d974cf647e7c347e8fa585,0.01564658,53.91451596,WETH,NaN,0x65519ef77c82302669b843a0b8b5f87077e286a2,1.0,completed,...,wormDTUJ6AWPNvk59vGQbDvGJmqbDTdgWgAqcLBCgUb,308730208,2024-12-20T19:16:02Z,2024-12-20T19:17:29.529738Z,[PORTAL_TOKEN_BRIDGE],0,EguycyPF6rQ4znP2KhcvKpLx1QcrYa4RsMdFAX6zxXHX,Solana,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,Ethereum
979,2024-12-20T18:59:35Z,Ethereum,0x6eb0d287a539aab2eb962de550fe5dda29b0fe52,50214,1946.45331624,AGA,NaN,0x3a7517341e11a338c015875559b6f76d564803ad,1.0,completed,...,NttADdCvGLUhukNyePei9CkmHoe6S9xjqgqfQv51PQg,308730249,2024-12-20T19:16:18Z,2024-12-20T19:17:29.624783Z,[NATIVE_TOKEN_TRANSFER],,BH3EDwnMJSef5aJS5CzADVsaZVEXkMP8gXfN5PgLS7TL,Solana,0x87B46212e805A3998B7e8077E9019c90759Ea88C,Ethereum
